In [1]:
TMVA::Tools::Instance();

TFile *input             = TFile::Open("higgs.root");
TMVA::DataLoader *loader = new TMVA::DataLoader("higgs");

TString outfilename = "TMVA_BDT_DNN.root";
TFile *output       = TFile::Open(outfilename, "RECREATE");
TMVA::Factory *factory   = new TMVA::Factory("TMVAClassification", output, "AnalysisType=Classification");

--- Factory                  : You are running ROOT Version: 6.07/07, Apr 1, 2016
--- Factory                  : 
--- Factory                  : _/_/_/_/_/ _|      _|  _|      _|    _|_|   
--- Factory                  :    _/      _|_|  _|_|  _|      _|  _|    _| 
--- Factory                  :   _/       _|  _|  _|  _|      _|  _|_|_|_| 
--- Factory                  :  _/        _|      _|    _|  _|    _|    _| 
--- Factory                  : _/         _|      _|      _|      _|    _| 
--- Factory                  : 
--- Factory                  : ___________TMVA Version 4.2.1, Feb 5, 2015
--- Factory                  : 


In [2]:
TTree *tsignal     = (TTree *) input->Get("TreeS");
TTree *tbackground = (TTree *) input->Get("TreeB");
Double_t signalWeight     = 1.0;
Double_t backgroundWeight = 1.0;
loader->AddSignalTree    (tsignal,     signalWeight);
loader->AddBackgroundTree(tbackground, backgroundWeight);
loader->AddVariable("lepton_pT",'F');
loader->AddVariable("lepton_eta",'F');
loader->AddVariable("lepton_phi",'F');
loader->AddVariable("missing_energy_magnitude",'F');
loader->AddVariable("missing_energy_phi",'F');
loader->AddVariable("jet_1_pt",'F');
loader->AddVariable("jet_1_eta",'F');
loader->AddVariable("jet_1_phi",'F');
loader->AddVariable("jet_1_b_tag",'F');
loader->AddVariable("jet_2_pt",'F');
loader->AddVariable("jet_2_eta",'F');
loader->AddVariable("jet_2_phi",'F');
loader->AddVariable("jet_2_b_tag",'F');
loader->AddVariable("jet_3_pt",'F');
loader->AddVariable("jet_3_eta",'F');
loader->AddVariable("jet_3_phi",'F');
loader->AddVariable("jet_3_b_tag",'F');
loader->AddVariable("jet_4_pt",'F');
loader->AddVariable("jet_4_eta",'F');
loader->AddVariable("jet_4_phi",'F');
loader->AddVariable("jet_4_b_tag",'F');
loader->AddVariable("m_jj",'F');
loader->AddVariable("m_jjj",'F');
loader->AddVariable("m_lv",'F');
loader->AddVariable("m_jlv",'F');
loader->AddVariable("m_bb",'F');
loader->AddVariable("m_wbb",'F');
loader->AddVariable("m_wwbb",'F');

--- DataSetInfo              : Dataset[higgs] : Added class "Signal"	 with internal class number 0
--- higgs                    : Add Tree TreeS of type Signal with 5829123 events
--- DataSetInfo              : Dataset[higgs] : Added class "Background"	 with internal class number 1
--- higgs                    : Add Tree TreeB of type Background with 5170877 events


In [3]:
TString dataString = "nTrain_Signal=5000000:"
                     "nTrain_Background=5000000:"
                     "nTest_Signal=100000:"
                     "nTest_Background=100000:"
                     "SplitMode=Random:"
                     "NormMode=NumEvents:"
                     "!V";
loader->PrepareTrainingAndTestTree("", "", dataString);

--- higgs                    : Preparing trees for training and testing...


In [4]:
factory->BookMethod(loader, TMVA::Types::kBDT, "BDT", "!V:NTrees=1000:MinNodeSize=2.5%:MaxDepth=3"
                                                      ":BoostType=AdaBoost:AdaBoostBeta=0.5"
                                                      ":UseBaggedBoost:BaggedSampleFraction=0.5"
                                                      ":SeparationType=GiniIndex:nCuts=20");


--- Factory                  : Booking method: BDT DataSet Name: higgs
--- DataSetFactory           : Dataset[higgs] : Splitmode is: "RANDOM" the mixmode is: "SAMEASSPLITMODE"
--- DataSetFactory           : Dataset[higgs] : Create training and testing trees -- looping over class "Signal" ...
--- DataSetFactory           : Dataset[higgs] : Weight expression for class 'Signal': ""
--- DataSetFactory           : Dataset[higgs] : Create training and testing trees -- looping over class "Background" ...
--- DataSetFactory           : Dataset[higgs] : Weight expression for class 'Background': ""
--- DataSetFactory           : Dataset[higgs] : Number of events in input trees (after possible flattening of arrays):
--- DataSetFactory           : Dataset[higgs] :     Signal          -- number of events       : 5829123  / sum of weights: 5.82912e+06
--- DataSetFactory           : Dataset[higgs] :     Background      -- number of events       : 5170877  / sum of weights: 5.17088e+06
--- DataSetFact

In [5]:
TString configString = "!H:V";
configString += ":Architecture=GPU";
configString += ":VarTransform=N";
configString += ":ErrorStrategy=CROSSENTROPY";
configString += ":WeightInitialization=XAVIERUNIFORM";
TString layoutString = "Layout=TANH|256,TANH|256,TANH|256,TANH|256,TANH|256,LINEAR";

TString trainingPhase1 = "TrainingStrategy="
                         "LearningRate=0.1,"
                         "ConvergenceSteps=20,"
                         "Momentum=0.9,"
                         "BatchSize=4096,"
                         "Regularization=L2,"
                         "WeightDecay=1e-5,"
                         "TestRepetitions=5";
TString trainingPhase2 = "LearningRate=0.01,"
                         "ConvergenceSteps=20,"
                         "Momentum=0.9,"
                         "BatchSize=4096,"
                         "Regularization=L2,"
                         "WeightDecay=1e-5,"
                         "TestRepetitions=5";
TString trainingPhase3 = "LearningRate=0.001,"
                         "ConvergenceSteps=20,"
                         "BatchSize=4096,"
                         "Momentum=0.5,"
                         "Regularization=L2,"
                         "WeightDecay=1e-5,"
                         "TestRepetitions=5";
TString trainingPhase4 = "LearningRate=0.0001,"
                         "ConvergenceSteps=20,"
                         "Momentum=0.2,"
                         "BatchSize=4096,"
                         "Regularization=L2,"
                         "WeightDecay=1e-5,"
                         "TestRepetitions=5";
TString trainingPhase5 = "LearningRate=0.00001,"
                         "ConvergenceSteps=20,"
                         "Momentum=0.0,"
                         "BatchSize=4096,"
                         "TestRepetitions=5";

configString += ":" + layoutString;
configString += ":" + trainingPhase1 + "|" + trainingPhase2 + "|" + trainingPhase3;
configString += "|" + trainingPhase4 + "|" + trainingPhase5;
TMVA::Types::EMVA method = TMVA::Types::kDNN;
factory->BookMethod(loader, method, "DNN", configString);

--- Factory                  : Booking method: DNN DataSet Name: higgs
--- <VERBOSE> Configurable             : Parsing option string: 
--- <VERBOSE> Configurable             : ... "!H:V:Architecture=GPU:VarTransform=N:ErrorStrategy=CROSSENTROPY:WeightInitialization=XAVIERUNIFORM:Layout=TANH|256,TANH|256,TANH|256,TANH|256,TANH|256,LINEAR:TrainingStrategy=LearningRate=0.1,ConvergenceSteps=20,Momentum=0.9,BatchSize=4096,Regularization=L2,WeightDecay=1e-5,TestRepetitions=5|LearningRate=0.01,ConvergenceSteps=20,Momentum=0.9,BatchSize=4096,Regularization=L2,WeightDecay=1e-5,TestRepetitions=5|LearningRate=0.001,ConvergenceSteps=20,BatchSize=4096,Momentum=0.5,Regularization=L2,WeightDecay=1e-5,TestRepetitions=5|LearningRate=0.0001,ConvergenceSteps=20,Momentum=0.2,BatchSize=4096,Regularization=L2,WeightDecay=1e-5,TestRepetitions=5|LearningRate=0.00001,ConvergenceSteps=20,Momentum=0.0,BatchSize=4096,TestRepetitions=5"
--- <VERBOSE> Configurable             : The following options are set:
--- <

In [6]:
factory->TrainAllMethods();

--- Factory                  :  
--- Factory                  : Train all methods for Classification ...
--- Factory                  : 
--- Factory                  : current transformation string: 'I'
--- Factory                  : Dataset[higgs] : Create Transformation "I" with events from all classes.
--- Id                       : Transformation, Variable selection : 
--- Id                       : Input : variable 'lepton_pT' (index=0).   <---> Output : variable 'lepton_pT' (index=0).
--- Id                       : Input : variable 'lepton_eta' (index=1).   <---> Output : variable 'lepton_eta' (index=1).
--- Id                       : Input : variable 'lepton_phi' (index=2).   <---> Output : variable 'lepton_phi' (index=2).
--- Id                       : Input : variable 'missing_energy_magnitude' (index=3).   <---> Output : variable 'missing_energy_magnitude' (index=3).
--- Id                       : Input : variable 'missing_energy_phi' (index=4).   <---> Output : variable 'mis

In [ ]:
factory->TestAllMethods();
factory->EvaluateAllMethods();

--- Factory                  : Test all methods...
--- Factory                  : Test method: BDT for Classification performance
--- BDT                      : Dataset[higgs] : Evaluation of BDT on testing sample (200000 events)


--- BDT                      : Dataset[higgs] : Elapsed time for evaluation of 200000 events: 0.1 sec       
--- Factory                  : Test method: DNN for Classification performance
--- DNN                      : Dataset[higgs] : Evaluation of DNN on testing sample (200000 events)
--- <FATAL> Norm                     : Transformation not yet created
***> abort program execution


In [ ]:
%jsroot on
TCanvas *c=factory->GetROCCurve(loader);
c->Draw();


 *** Break *** segmentation violation
